# Lab 2: Exploratory Data Analysis (Python, Pandas & matplotlib)

In this lab 2, we are going to learn how to explore data using Python. Namely, we will work with Pandas and matplotlib, for data filtering and grouping, and visualization respectively. Provided VM is recommeded for this lab.

## Setup

#### This Notebook

Download this notebook so you can edit it. (If you are viewing it via nbviewer.ipython.org, then use the link in the upper right corner.) To edit this notebook, in your VM terminal, type "ipython notebook" and in your prompted brower, click the notebook file to open and edit it.

#### Data

We'll use the sampled data from the log file in lab 1, download the file 'wc_day6_1_sample.csv' [here](https://ufl.instructure.com/courses/344204/files/folder/Labs/Lab%202) and put it in your VM (To enable the 'Drag'n Drop' for ease of transfering files to/from VM, go to Device -> Drag 'n Drop -> Bidirectional)

## Pandas

The two main data structures used by pandas are

 * Series: one-dimensional collections of any data type.
 * DataFrames: two-dimensional data structures similar to a database table.

At first, we need to import the libraries we need. In the code cell below, we will import two libraries, pandas for data processing and PyLab for plotting.

To run the code in the cell below, select the cell, then press 'ctrl + enter', or hit the 'play' button above.

In [ ]:
import pylab
import pandas as pd

## The Basics

To create our first DataFrame using pandas, run:

In [ ]:
df = pd.DataFrame( { 'a' : [1, 2, 3, 4], 'b': [ 'w', 'x', 'y', 'z'] })

This is going to create a table with two columns and four rows, we can inspect the DataFrame by typing the name on the object:

In [ ]:
df

For more detailed information about the schema of a DataFrame we can use the info function:

In [ ]:
df.info()

To check the first few rows we can use head and the last few rows tail functions

In [ ]:
df.head(2)

df.tail(2)

To access any range of rows in the DataFrame we can use array-like indexes, for example, the following will retrieve rows 1 and 2 from our DataFrame:

In [ ]:
df[1:3]

For the following exercises we will use the dataset mentioned in setup.

After you download this CSV file, we can import it into a DataFrame, assuming the file is in the path /home/datascience/

In [ ]:
log_df = pd.read_csv("/home/datascience/labs/notebooks/wc_day6_1_sample.csv",
                     names=['ClientID', 'Date', 'Time', 'URL', 'ResponseCode', 'Size'],
                     na_values=['-'])

Using read_csv function, we created a new DataFrame with column names passed as a list in the names parameter and null or non existing values represented with "-" in the na_values parameter.

## More complex operations

#### Row filtering

To create row filters, we use lists of Boolean values that evaluate a condition:

In [ ]:
is_may1st = log_df['Date'] == '01/May/1998'

This returns a list of True/False values for each row in **log_df** that evaluate the given condition. To filter using this list:

In [ ]:
may1_df = log_df[is_may1st]

This can be done more concisely:

In [ ]:
may1_df = log_df[log_df['Date'] == '01/May/1998']

#### Column filtering

For column filters, we use the name of the columns that we want to keep passes as a string array:

In [ ]:
url_codes = log_df[['URL', 'ResponseCode']]

#### Grouping

For grouping by a column, i.e. dividing all the rows into groups of the same value in that column, we use **groupby** which does not return a DataFrame but a **DataFrameGroupBy** object. The **DataFrameGroupBy** object is a dictionary-like object where keys are the distinct values in the grouping column, and values are DataFrames containnig th remaining columns

In [ ]:
grouped = log_df.groupby('ResponseCode')

In [ ]:
grouped.get_group(200)

In [ ]:
# group by multiple columns
multi_grouped = log_df.groupby(['ResponseCode', 'Date'])

Similarly, we can apply operations to each group such as:

In [ ]:
multi_grouped.get_group((200, '30/Apr/1998')).info()

In [ ]:
grouped.size()
# returns a panda Series(1-d array with the groupby value(s)as index, count of each group as data values.)

In [ ]:
grouped.sum(), grouped.mean(), grouped.median()

## Basic Statistics

Pandas can also provide useful insights of numerical columns in a DataFrame. The function describe will provide basic statistics such as count, mean, standard deviation, etc.

In [ ]:
log_df.describe()

## Visualization

Now lets visualize our data using PyLab (this step requires to have matplotlib installed, already installed in the virtual machine)

#### Pie Chart

In [ ]:
# The sizes of each group by ResponseCode
grouped.size()

In [ ]:
%matplotlib inline
# show the percentage of each response code
import matplotlib.pyplot as plt
grouped.size().plot(kind='pie', legend=True)

#### Bar Chart

In [ ]:
# Or show as bar chart
grouped.size().plot(kind='bar')

In [ ]:
# Since the diferences between the counts are huge, use logy y-axis
ax = grouped.size().plot(kind='bar')
ax.set_yscale('log', nonposy='clip')# the 'noposy='clip'' is to replace the invalid log(0) with a very small positive one
                                  # while doing the log-scale transition.

Now let's check how requests changes during different hour of the day on '01/May/1998'

#### Line Plot

In [ ]:
may1_df = log_df[log_df['Date'] == '01/May/1998']
may1_df.loc[:,('DateTime')] = pd.to_datetime(may1_df.apply(lambda row: row['Date'] + ' ' + row['Time'], axis=1))
hour_grouped = may1_df.groupby(lambda x: may1_df['DateTime'][x].hour)

In [ ]:
# Let's check how many requests were made during each hour of the day on '01/May/1998' 
hour_grouped.size()

In [ ]:
# and how much traffic the server handled during each hour:
hour_grouped['Size'].sum()

In [ ]:
# plot the changes of traffic on different hours of the day.
ax = hour_grouped.size().plot()
ax.set_ylabel("# Requests")
ax.set_xlabel("Hour of the day")
ax.set_title("# Request changes in a day")

In [ ]:
# Let's show the # requests and size of traffic in a single graph:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
x = hour_grouped.size().index

ax1.plot(x, hour_grouped.size(), 'g-')
ax2.plot(x, hour_grouped['Size'].sum(), 'r-')

ax1.set_xlabel('Hour of the day')
ax1.set_ylabel('# Requests', color='g')
ax2.set_ylabel('Size of traffic handled', color='r')

## Now it's your turn:

### Homework 2

NOTE: You don't have to submit anything for this homework.

Questions:
1. How many requests that were sent on '30/Apr/1998' and had HTTP return code 404?
2. Generate a plot of the number of **distinct** users of the site every hour on 01/May/1998 using bar chart.
3. Generate a line plot of the number of **distinct** users vs  size of traffic of the site every hour on 01/May/1998, is there any correlation? (hint: use "hour_grouped['ClientID'].nunique()" to get the number of distinct users)
5. We wish to see if there is any correlation between client-ids and hours of the day at which they visit the website. Get 100 client ids from the dataset and generate a scatter plot that shows the hours of the day these clients sent requests. Hint: df.plot(kind='scatter', x='a', y='b'); and df['Column'].unique()
6. The log file used in the lab was from one day of the WorldCup. Lets apply our analysis to Jul/24 and Jul/25 in the [log(wc_day91_1.log.tar.bz2)](https://ufl.instructure.com/courses/344204/files/folder/Labs/Lab%202). Repeat exercises 3 and 4 with it. How similar or different are the results? Hint: You can use UNIX command line tools from Lab 1 to first get a csv file and then load it into Pandas.
